Deep Learning Assignment 2 Pujan

code reference: https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

In [38]:

import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm


In [39]:

def get_device():
    return torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

get_device()


device(type='cuda', index=0)

In [40]:

class PujanNet(nn.Module):
    def __init__(self, data_set_name, classes, batch_size, download_path):
        super().__init__()

        transform = transforms.Compose(
            [transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

        if data_set_name == "cifar_10":
            self.trainset = torchvision.datasets.CIFAR10(root=download_path, train=True, download=True, transform=transform)
            self.trainloader = torch.utils.data.DataLoader(self.trainset, batch_size=batch_size, shuffle=True, num_workers=2)

            self.testset = torchvision.datasets.CIFAR10(root=download_path, train=False, download=True, transform=transform)
            self.testloader = torch.utils.data.DataLoader(self.testset, batch_size=batch_size, shuffle=False, num_workers=2)

        self.classes = classes

        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def display_input_images(self, number_of_images):
        # get some random training images
        dataiter = iter(self.trainloader)
        images, labels = dataiter.next()

        # show images
        imshow(torchvision.utils.make_grid(images))
        # print labels
        print(' '.join(f'{self.classes[labels[j]]:5s}' for j in range(number_of_images)))

    def display_ground_truth_images(self, number_of_images):
        dataiter = iter(self.testloader)
        images, labels = dataiter.next()

        # print images
        imshow(torchvision.utils.make_grid(images))
        print('GroundTruth: ', ' '.join(f'{classes[labels[j]]:5s}' for j in range(4)))


    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def train(self, epoch_number, learning_rate, momentum, criterion, optimizer_name):
        optimizer = optim.SGD(self.parameters(), lr= learning_rate, momentum=momentum)
        # if optimizer_name == "sgd": 

        for epoch in range(epoch_number):  # loop over the dataset multiple times
            running_loss = 0.0
            for i, data in enumerate(self.trainloader, 0):
                # get the inputs; data is a list of [inputs, labels]
                # inputs, labels = data
                inputs, labels = data[0].to(get_device()), data[1].to(get_device())

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward + backward + optimize
                outputs = self(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                # print statistics
                running_loss += loss.item()
                if i % 2000 == 1999:    # print every 2000 mini-batches
                    print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
                    running_loss = 0.0

        print('Finished Training')

    def predictions(self, input_images):
        outputs = self(input_images)
        _, predicted = torch.max(outputs, 1)

        print('Predicted: ', ' '.join(f'{classes[predicted[j]]:5s}' for j in range(4)))

    def accuracy(self):
        correct = 0
        total = 0
        # since we're not training, we don't need to calculate the gradients for our outputs
        with torch.no_grad():
            for data in self.testloader:
                # images, labels = data
                images, labels = data[0].to(get_device()), data[1].to(get_device())
                # calculate outputs by running images through the network
                outputs = self(images)
                # the class with the highest energy is what we choose as prediction
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

    def accuracy_in_each_class(self):
    # prepare to count predictions for each class
        correct_pred = {classname: 0 for classname in classes}
        total_pred = {classname: 0 for classname in classes}

        # again no gradients needed
        with torch.no_grad():
            for data in self.testloader:
                # images, labels = data
                images, labels = data[0].to(get_device()), data[1].to(get_device())
                outputs = self(images)
                _, predictions = torch.max(outputs, 1)
                # collect the correct predictions for each class
                for label, prediction in zip(labels, predictions):
                    if label == prediction:
                        correct_pred[classes[label]] += 1
                    total_pred[classes[label]] += 1

        # print accuracy for each class
        for classname, correct_count in correct_pred.items():
            accuracy = 100 * float(correct_count) / total_pred[classname]
            print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')



In [41]:

criterion = nn.CrossEntropyLoss()
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
pujan_net = PujanNet(data_set_name = "cifar_10", classes = classes, batch_size = 4, download_path = "./data")
pujan_net = pujan_net.to(get_device())

pujan_net.train(epoch_number = 2, learning_rate = 0.01, momentum = 0.9, criterion = criterion, optimizer_name = "sgd")

pujan_net.accuracy()

Files already downloaded and verified
Files already downloaded and verified
[1,  2000] loss: 2.104
[1,  4000] loss: 1.975
[1,  6000] loss: 1.954
[1,  8000] loss: 1.987
[1, 10000] loss: 1.979
[1, 12000] loss: 1.961
[2,  2000] loss: 1.973
[2,  4000] loss: 1.978
[2,  6000] loss: 1.998
[2,  8000] loss: 2.023
[2, 10000] loss: 2.146
[2, 12000] loss: 2.029
Finished Training
Accuracy of the network on the 10000 test images: 25 %


In [ ]:
# Experiments
# 1. CNN Architectures (How to choose CNN architecture ????)
# 2. Number of Layers ??
# 1. Loss
# 2. Optimizer
# 3. Epoch
# 4. Learning rate
# 5. Data augmentation
